<a href="https://colab.research.google.com/github/33quitykubby/SIGNATE_Practice/blob/main/Tommy_G/SGN_Casting_20210918_WandB_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **【練習問題】鋳造製品の欠陥検出**  
https://signate.jp/competitions/406/data

---

Pytorchを使った2値分類  
モデル：EfficientNet B0  
Weights and Biasesの導入テスト

参考  
https://axa.biopapyrus.jp/deep-learning/pytorch/simple-cnn-classification.html

Dataloader  
https://pystyle.info/pytorch-how-to-create-custom-dataset-class/  
https://qiita.com/kumonk/items/0f3cad018cc9aec67a63

交差検証  
https://qiita.com/ground0state/items/ad879a84bf946ef94da8  

学習  
https://hntk03.hatenablog.com/entry/2019/08/27/112435

コンペ：prob space 宗教画テーマの分類  
トピック EfficientNetB0使ったベースライン紹介(skywalker氏)  
https://comp.probspace.com/competitions/religious_art/submissions\\

## Weights and Biases  
---
https://wandb.ai/home

参考  
pytorch + WandB  
https://theaisummer.com/weights-and-biases-tutorial/

colab + WandB  
https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Simple_PyTorch_Integration.ipynb#scrollTo=sAZBTFNQq0ys

# マウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setting

In [2]:
!nvidia-smi

Wed Sep 22 11:44:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# !pip uninstall albumentations -y
# !pip install albumentations==0.4.5 -q
!pip install albumentations==0.5.0 -q

     |████████████████████████████████| 70 kB 3.8 MB/s 
     |████████████████████████████████| 37.1 MB 50 kB/s 
     |████████████████████████████████| 948 kB 40.9 MB/s 


In [4]:
!pip install timm -q

     |████████████████████████████████| 376 kB 5.2 MB/s 


# Weights and Biases セットアップ
---
https://wandb.ai/home

In [5]:
!pip install wandb --upgrade

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 139 kB 45.7 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 180 kB 50.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=5cafafc89d672431441ae3a767a6630d0bb4b052ddaed683130459e4408ea946
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=6232422235a69f9290dd57a1ddbe38e44aeb3362b7a4cbef1f932f7ad7bd32a3
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [6]:
import wandb

# 認証コードを入力する必要がある
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
# Weight and Biasesのプロジェクト名
WAB_PROJECT_NAME = "test_casting_2"

#
WAB_ACCONT = "tommyg"

# パス指定

In [8]:
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/'

OUTPUT_DIR = '/content/drive/Othercomputers/myPC/GoogleDrive/OutputFile/Signate_Casting/'


# **import**

In [9]:
import warnings
warnings.simplefilter('ignore')
import os
import gc
gc.enable()
import sys
import glob
import time
import random
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as Func
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.model_selection import GroupKFold, StratifiedKFold
from datetime import datetime
from tqdm.autonotebook import tqdm as tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score
import timm

import datetime

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path

In [10]:
  !rm /etc/localtime
  !ln -s /usr/share/zoneinfo/Asia/Tokyo /etc/localtime
  !date

#終了時刻
time_of_begin = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))

Wed Sep 22 20:45:18 JST 2021


# 乱数シード固定

In [11]:
def seed_everything(seed:int==42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(32)

In [12]:

# wandb.init(project='test', entity=WAB_ACCONT)

# データ読み込み

## MyDataset(未使用)

In [13]:
import torchvision.transforms as transforms

class MyDataset(torch.utils.data.Dataset):

  def __init__(self, imageSize, dir_path, transform=None):
    self.transforms = transforms.Compose([
                                          transforms.Resize(imageSize),
                                          transforms.ToTensor(),
    ])

    train_data_dir = dir_path + "train_data/"
    self.image_paths = [str(p) for p in Path(train_data_dir).glob("*.jpeg")]
    self.data_num = len(self.image_paths) #lenの返り値

    # ラベル用のデータ読み込み
    self.train_labels = pd.read_csv(dir_path + "train.csv", index_col='id')

  def __len__(self):
    return self.data_num

  def __getitem__(self, idx):
    p = self.image_paths[idx]
    image = Image.open(p)

    print(p)

    if self.transform:
      out_data = self.transform(image)

    #ラベル取り出し
    str_p = p.rsplit('/', 1)[1]    
    label = self.train_labels['target'][str_p]

    return out_data, out_label


## TrainDataset

In [14]:
import torchvision.transforms as transforms

class TrainDataset(torch.utils.data.Dataset):

  def __init__(self, imageSize, dir_path, index, transform=None):
    self.transform = A.Compose([
                                          A.Resize(imageSize,imageSize),
                                          A.VerticalFlip(p=0.5),
                                          A.HorizontalFlip(p=0.5),
                                          # A.Normalize(),
                                          ToTensorV2(),
    ])

    train_data_dir = dir_path + "train_data/"
    self.image_paths = [str(p) for p in Path(train_data_dir).glob("*.jpeg")]

    self.index = index
    # print(index[1])
    # print(index[249])

    self.data_num = len(self.index) #lenの返り値

    # ラベル用のデータ読み込み
    self.train_labels = pd.read_csv(dir_path + "train.csv", index_col='id')

  def __len__(self):
    return self.data_num

  def __getitem__(self, idx):
    current_index = self.index[idx]
    # print(idx)
    # print(current_index)

    p = self.image_paths[current_index]
    image = Image.open(p)
    image = np.array(image)

    img = self.transform(image=image)
    out_data = img['image']

    #ラベル取り出し
    str_p = p.rsplit('/', 1)[1]    
    out_label = self.train_labels['target'][str_p]

    return {
            'image' : torch.tensor(out_data, dtype=torch.float),
            'target' : torch.tensor(out_label, dtype=torch.float).long(),}


## TestDataset

In [15]:
class TestDataset(torch.utils.data.Dataset):

  def __init__(self, imageSize, dir_path, transform=None):
    self.transform = A.Compose([
                                          A.Resize(imageSize,imageSize),
                                          # A.Normalize(),
                                          ToTensorV2(),
    ])


    train_data_dir = dir_path + "test_data/"
    self.image_paths = [str(p) for p in Path(train_data_dir).glob("*.jpeg")]

    self.data_num = len(self.image_paths) #lenの返り値

  def __len__(self):
    return self.data_num

  def __getitem__(self, idx):
    # print(self.image_paths[idx])
    p = self.image_paths[idx]
    image = Image.open(p)
    image = np.array(image)

    img = self.transform(image=image)
    out_data = img['image']

    return {
            'image' : torch.tensor(out_data, dtype=torch.float),}


# 全データ取得

In [16]:
train_labels = pd.read_csv(BASE_PATH + "train.csv", index_col='id')

print(train_labels.shape)
train_labels

(250, 1)


,target
id,
cast_def_0_335.jpeg,0
cast_def_0_323.jpeg,0
cast_def_0_61.jpeg,0
cast_def_0_163.jpeg,0
cast_def_0_175.jpeg,0
...,...
cast_ok_0_346.jpeg,1
cast_ok_0_307.jpeg,1
cast_ok_0_242.jpeg,1


# Model

In [17]:
# モデル
MODEL_DEFINE = 'efficientnet_b0'

# クラス
CLASSES = 2

# エポック
EPOCHS = 10


In [18]:
class Model(nn.Module):    
    def __init__(self, pretrained=False):
        super().__init__()
        self.modelbase = timm.create_model(MODEL_DEFINE, 
                                          pretrained=pretrained,
                                          in_chans=3)        
        self.n_features = self.modelbase.classifier.in_features
        self.modelbase.classifier = nn.Linear(self.n_features, CLASSES)

    def forward(self, x):
        output = self.modelbase(x)
        output = Func.log_softmax(output, dim = 1)
        return output


In [19]:
momdel = Model()

# model

# 学習

In [20]:
# 学習率
LEARNING_RATE = 1e-3

# 減衰率
WEIGHT_DECAY = 1e-5

# GPU/CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ベストケースのモデル情報の格納場所
MODEL_DIR = BASE_PATH + "model_data/"

# fold数
FOLDS = 3

## 学習関数

In [21]:
def func_train(model, device, optimizer, loss_function, train_loader):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    for batch_idx,  train in enumerate(train_loader):
        images = train['image'].to(device)
        labels = train['target'].to(device)
        
        optimizer.zero_grad() 
        preds = model(images)
        
        loss = loss_function(preds, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    train_loss = running_loss / len(train_loader)
    
    return train_loss

## valid関数

In [22]:

def func_valid(model, device, loss_function, dataloader ):    
    model.eval()
    running_loss = 0
    test_preds = []

    for batch_idx, valid in enumerate(dataloader):
        images = valid['image'].to(device)
        labels = valid['target'].to(device)
        preds = model(images)     
        
        loss = loss_function(preds, labels) 
        running_loss += loss.item()

    valid_loss = running_loss / len(dataloader)   
    return valid_loss

## テスト関数

In [23]:

def func_test(model, device, dataloader ):    
    model.eval()        
    final_loss = 0
    test_preds = []

    for step, test in enumerate(dataloader):
        print("stap=",step)    
        images = test['image'].to(device)
        preds = model(images)            
        test_preds.append(preds.sigmoid().detach().cpu().numpy())                
    
    test_preds = np.concatenate(test_preds)    
    return test_preds

## 学習と推論の実行（メインの関数）

In [24]:
def do_train_predict(config=None):

    # kf = StratifiedKFold(n_splits=3)
    from sklearn.model_selection import KFold

    kf = KFold(n_splits=FOLDS, shuffle=True)


    # テストデータの数を知りたい
    dir = BASE_PATH + "test_data/"
    test_len = sum(os.path.isfile(os.path.join(dir,name)) for name in os.listdir(dir))

    # pred初期化
    pred = np.zeros((test_len, CLASSES))

    cv = 0
    for _fold, (train_index, valid_index) in enumerate(kf.split(train_labels)):

      wandb.init(project=WAB_PROJECT_NAME, entity=WAB_ACCONT, config=config, group='casting_A', job_type="eval")
      config = wandb.config    

      # batch_size = 16
      batch_size = config.batch_size

      # print(train_index)
      # print(valid_index)  

      # loss最小値の初期化(とりあえず100)
      min_loss = 100


      train_dataset = TrainDataset(224, dir_path=BASE_PATH, index = train_index)
      train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)

      valid_dataset = TrainDataset(224, dir_path=BASE_PATH, index = valid_index)
      valid_dataloader = DataLoader(valid_dataset, batch_size, shuffle=False)

      model = Model(pretrained=True)
      model.to(DEVICE)
      optimizer = torch.optim.Adam(model.parameters(),
                                    # lr=LEARNING_RATE,
                                    lr = config.learning_rate,
                                    weight_decay=WEIGHT_DECAY)
      
      loss_function = nn.CrossEntropyLoss()

      start_time = time.time()
      end_time = start_time

      # 学習
      # EPOCHS = 2 # for test
      for epoch in range(EPOCHS):
        start_time = time.time()

        train_loss = func_train(model, DEVICE, optimizer, loss_function, train_dataloader)
        valid_loss = func_valid(model, DEVICE, loss_function, valid_dataloader)

        if valid_loss < min_loss:
            min_loss = valid_loss            
            MODEL_PATH = f"{MODEL_DIR}best_{MODEL_DEFINE}_fold{_fold+1}.pth"
            torch.save(model.state_dict(), MODEL_PATH)
        # else:
        #     continue

        end_time = time.time()
        print(f"FOLD: {_fold+1} | EPOCH:{epoch+1:02d} | train_loss:{train_loss:.6f} | valid_loss:{valid_loss:.6f} | time:{end_time-start_time:.1f}s ")
        wandb.log({'epoch': epoch + 1, 'loss': valid_loss})

      print("final valid_loss : ",valid_loss )

      model.load_state_dict(torch.load(MODEL_PATH))
      test_dataset = TestDataset(224, dir_path=BASE_PATH)
      test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

      pred_list = func_test(model, DEVICE, test_dataloader)

      # print(pred_list)

      pred += pred_list/FOLDS

      wandb.finish()


## Weights and Biases処理

In [25]:
import math

sweep_config = {
                'method': 'random',
                # 'metric': {'goal': 'minimize', 'name': 'loss'},
                'parameters': {
                    'batch_size': {
                        'distribution': 'q_log_uniform',
                        'max': math.log(32),
                        'min': math.log(16),
                        'q': 1
                    },
                #     'epochs': {'value': 5},
                #     'fc_layer_size': {'values': [128, 256, 512]},
                    'learning_rate': {'distribution': 'uniform',
                                      'max': 1e-3,
                                      'min': 1e-4},
                    # 'optimizer': {'values': ['adam', 'sgd']
                }
 }

In [26]:
sweep_id = wandb.sweep(sweep_config, project=WAB_PROJECT_NAME)

Create sweep with ID: 6rkvcus8
Sweep URL: https://wandb.ai/tommyg/test_casting_2/sweeps/6rkvcus8


In [27]:
wandb.agent(sweep_id, function=do_train_predict, count=5)

wandb: Agent Starting Run: 2497wpzh with config:
wandb: 	batch_size: 19
wandb: 	learning_rate: 0.0005552131735236642
wandb: Currently logged in as: tommyg (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


FOLD: 1 | EPOCH:01 | train_loss:0.424881 | valid_loss:4.964986 | time:64.2s 
FOLD: 1 | EPOCH:02 | train_loss:0.132970 | valid_loss:0.260991 | time:4.2s 
FOLD: 1 | EPOCH:03 | train_loss:0.162685 | valid_loss:0.053393 | time:4.2s 
FOLD: 1 | EPOCH:04 | train_loss:0.052349 | valid_loss:0.164951 | time:4.0s 
FOLD: 1 | EPOCH:05 | train_loss:0.037040 | valid_loss:0.034552 | time:4.2s 
FOLD: 1 | EPOCH:06 | train_loss:0.021182 | valid_loss:0.009307 | time:4.2s 
FOLD: 1 | EPOCH:07 | train_loss:0.012492 | valid_loss:0.004485 | time:4.2s 
FOLD: 1 | EPOCH:08 | train_loss:0.028461 | valid_loss:0.001749 | time:4.1s 
FOLD: 1 | EPOCH:09 | train_loss:0.030640 | valid_loss:0.017378 | time:4.0s 
FOLD: 1 | EPOCH:10 | train_loss:0.012001 | valid_loss:0.295384 | time:4.0s 
final valid_loss :  0.29538409506585594
stap= 0
stap= 1
stap= 2
stap= 3
stap= 4
stap= 5


epoch,10
loss,0.29538


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▁▁▁▁▁▁▁


...

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▂▂▂▁▁▁▁▁▁


wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


FOLD: 3 | EPOCH:01 | train_loss:0.345273 | valid_loss:20.925450 | time:4.1s 
FOLD: 3 | EPOCH:02 | train_loss:0.187301 | valid_loss:5.197004 | time:4.0s 
FOLD: 3 | EPOCH:03 | train_loss:0.109656 | valid_loss:0.141241 | time:4.1s 
FOLD: 3 | EPOCH:04 | train_loss:0.040297 | valid_loss:0.139246 | time:4.1s 
FOLD: 3 | EPOCH:05 | train_loss:0.035800 | valid_loss:0.066142 | time:4.0s 
FOLD: 3 | EPOCH:06 | train_loss:0.018788 | valid_loss:0.051152 | time:4.1s 
FOLD: 3 | EPOCH:07 | train_loss:0.036842 | valid_loss:0.035833 | time:4.0s 
FOLD: 3 | EPOCH:08 | train_loss:0.022405 | valid_loss:0.048973 | time:3.9s 
FOLD: 3 | EPOCH:09 | train_loss:0.021038 | valid_loss:0.025999 | time:4.1s 
FOLD: 3 | EPOCH:10 | train_loss:0.008920 | valid_loss:0.009881 | time:4.0s 
final valid_loss :  0.00988102561677806
stap= 0
stap= 1
stap= 2
stap= 3
stap= 4


epoch,10
loss,0.00988


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▁▁▁▁▁▁▁▁


In [28]:
# print (pred)


# 結果出力


## サンプルcsv読み込み

In [29]:
test_samples = pd.read_csv(BASE_PATH + "sample_submission.csv", header=None)

pred_df = test_samples.set_axis(['filename', 'y'], axis=1)

print(pred_df.shape)
pred_df


(100, 2)


,filename,y
0,19871.jpeg,0
1,29934.jpeg,0
2,39826.jpeg,0
3,49888.jpeg,0
4,59918.jpeg,0
...,...,...
95,969983.jpeg,0
96,979891.jpeg,0
97,989921.jpeg,0
98,999921.jpeg,0


## ファイル名をテストデータで入力している順にする

テストディレクトリの読み出し順

In [30]:
test_data_dir = BASE_PATH + "test_data/"
test_image_paths = [str(p) for p in Path(test_data_dir).glob("*.jpeg")]

print(test_image_paths)

for i in range(len(test_image_paths)):
  test_image_paths[i] = test_image_paths[i].rsplit('/', 1)[1]

pred_df['filename'] = test_image_paths

pred_df

['/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/229958.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/559880.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/699970.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/609988.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/929917.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/89914.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/79963.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/779897.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/549992.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/709966.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/589947.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/949956.jpeg', '/content/drive/MyDrive/Colab Notebooks/SIGNATE_鋳造/test_data/269945.jpeg', '/content/drive/MyDrive/Co

,filename,y
0,229958.jpeg,0
1,559880.jpeg,0
2,699970.jpeg,0
3,609988.jpeg,0
4,929917.jpeg,0
...,...,...
95,819873.jpeg,0
96,769946.jpeg,0
97,59918.jpeg,0
98,189978.jpeg,0


## 学習結果をセット

---

今はコメントオフ

In [31]:
# pred_df['y'] = np.argmax(pred, axis=1).astype('int')

# pred_df

In [32]:
# for i in range(CLASSES):
#     pred_df[f'pred_{i}'] = pred[:, i]

# now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))

# #詳細
# str_detail_file = OUTPUT_DIR  + 'SGN_Casting_detail_EffB0_' +  now.strftime('%m%d%H%M') +'.csv'
# pred_df.to_csv(str_detail_file, index=False)

# str_file = OUTPUT_DIR  + 'SGN_Casting_submit__EffB0_' +  now.strftime('%m%d%H%M') +'.csv'
# pred_df[['filename', 'y']].to_csv(str_file, header=False, index=False)

# display(pred_df.head())

# 終わり

In [33]:
#終了時刻
time_of_finish = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print("finish_time=",time_of_finish)
print("total_time=",time_of_finish-time_of_begin)

finish_time= 2021-09-22 20:59:06.738763+09:00
total_time= 0:13:47.907965
